In [ ]:
%matplotlib inline

import sys,os, warnings, time
import numpy as np
import matplotlib.cm
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib import colors
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import ats_xdmf
import datetime as dt
import colors as clrs

%config InlineBackend.figure_format = 'retina'

warnings.filterwarnings("ignore")

sns.set(style = 'ticks', font_scale=1.5)

print(os.environ['ATS_SRC_DIR'])

# sys.path.append("../")
from my_utils import my_utils

In [ ]:
def absmax(a):
    mi = np.min(a)
    ma = np.max(a)
    if np.abs(mi) > np.abs(ma):
        m = mi
    else:
        m = ma
    return m

class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, vcenter=None, clip=False):
        self.vcenter = vcenter
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.vcenter, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))


def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
regions=['cd','cw','wd','ww']
r_labels = ["cold-dry", "cold-wet", "warm-dry", "warm-wet"]
colors = ['darkorange','navy', 'darkorange', 'navy']
ls = ['-', '-', '--', '--']

tex = '02peat'
scenarios = ['c', 'i']

dr = pd.date_range(start='01/01/2017', end='12/31/2021', freq='D')
months = mdates.MonthLocator(interval = 4)

dr2 = pd.date_range(start='01/01/2017', end='01/01/2018', freq='D')
months2 = mdates.MonthLocator(interval = 4)
basepath = os.path.abspath(os.path.join(os.getcwd()))

plotpath = basepath+'/plots/'
dtFmt2 = mdates.DateFormatter('%b')

In [ ]:
alt = np.zeros((2,len(regions)))
fig,ax = plt.subplots(figsize=(10,3))
for k in range(len(regions)):
    print(regions[k])
    al = np.zeros((366,2))
    td_all = np.zeros((366,2))
    for j in range(len(scenarios)):

        directory=basepath + '/regions/'+regions[k] + '/'+tex+'/' + scenarios[j]

        times, z, dati = my_utils.get_variable_field(directory, 'temperature', time_unit='d')

        for i,t in enumerate(times[:365]):
            temp = dati[i,:]
            td = np.argmax(temp > 273.15)
            if td == 0:
                z_td = 0
            else:
                z_td = z[td]
            al[i,j] = z_td

    if tex == '02peat':
        ax.plot(dr[135:280],al[135:280,0], color = colors[k], linestyle=ls[k], label=r_labels[k])
    else:
        ax.plot(dr[135:290],al[135:290,0], color = colors[k], linestyle=ls[k], label=r_labels[k])
    
    ax.fill_between(dr[135:280],al[135:280,0],al[135:280,1], color= colors[k], alpha=0.4, edgecolor="none")

    print("Max ALT in REF is: " + str(np.min(al,axis=0)[0]))
    print("Max ALT in HR is: " + str(np.min(al,axis=0)[1]))
    print("REF thaw begins: " + str(dr2[al[:,0]<0][0].strftime("%B %d")))
    print("REF freezeup begins: " + str(dr2[al[:,0]<0][-1].strftime("%B %d")))
    print("HR thaw begins: " + str(dr2[al[:,1]<0][0].strftime("%B %d")))
    print("HR freezeup begins: " + str(dr2[al[:,1]<0][-1].strftime("%B %d")))
    
    ref_alt = np.min(al,axis=0)[0]
    hr_alt = np.min(al,axis=0)[1]
    
    alt[0,k] = ref_alt
    alt[1,k] = hr_alt

ax.axvline(dt.datetime(2017, 6, 15), color = 'gainsboro', alpha = 0.5,lw=3)
ax.axvline(dt.datetime(2017, 7, 15), color = 'gainsboro', alpha = 0.5,lw=3)
ax.axvline(dt.datetime(2017, 8, 15), color = 'gainsboro', alpha = 0.5,lw=3)
ax.text(0.01, 0.99, 'c) thaw depth', transform=ax.transAxes, va='top', ha='left', fontsize=15, fontweight='bold')
ax.set_ylabel('thawdepth [m]')

if tex == '02peat':
    ax.set_ylim(-0.8,0.1)
else:
    ax.set_ylim(-1.2,0.15)

ax.xaxis.set_major_formatter(dtFmt2)

ax.legend(frameon=False)

plt.tight_layout()

In [ ]:
# fig.savefig(plotpath + 'thawdepth.svg', bbox_inches='tight', dpi=300)
fig.savefig(plotpath + 'thawdepth.pdf', bbox_inches='tight', dpi=300)
# fig.savefig(plotpath + 'thawdepth_transparent.svg', bbox_inches='tight', dpi=300, transparent=True)